### Sign in with your credential and get a hanlde

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id="",
    resource_group_name="",
    workspace_name=""
)

### Create data asset

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = './data/scores.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Principle components dataset of CpG site methylation betas",
    name="dataset",
    version="1"
)

ml_client.data.create_or_update(my_data)

### Create the environment

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="pytorch/pytorch:latest",
    conda_file="./env.yml",
    name="pytorch-methylation",
    description="Environment for methylation training",
    version = '1'
)
ml_client.environments.create_or_update(env_docker_conda)

### Configure the training job with hyperparameters tunning

In [2]:
from azure.ai.ml import command

# configure command job as base
job = command(
    code="./",
    command="python train.py --training_data ./scores.csv  --learning_rate ${{inputs.learn_rate}} --beta_one ${{inputs.beta_one}} --beta_two ${{inputs.beta_two}} --epsilon ${{inputs.eps}} --weight_decay ${{inputs.decay}} --epochs ${{inputs.epochs}} --batch_size ${{inputs.batch}} --testset_n 10",
    inputs={
        "learn_rate": 1e-3, "beta_one": 0.9, "beta_two": 0.99, "eps": 1e-8, "decay": 3e-4, "epochs": 1000, "batch": 64
    },
    environment="pytorch-methylation",
    compute="aml-cluster",
    )

In [3]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    learn_rate=Choice(values=[1e-5, 1e-4, 1e-3, 5e-3]),
    beta_one=Choice(values=[0.9, 0.85, 0.95]),
    beta_two=Choice(values=[0.99, 0.995]),
    eps=Choice(values=[1e-8, 1e-7]),
    decay=Choice(values=[1e-5, 1e-4, 5e-4]),
    epochs=Choice(values=[100, 250, 500]),
    batch=Choice(values=[32, 64, 128])
)

### Configure the termination parameters and run the job

In [ ]:
from azure.ai.ml.sweep import BanditPolicy

sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="bayesian",
    primary_metric="MSE",
    goal="Minimize",
)

sweep_job.early_termination = BanditPolicy(
    slack_amount = 50,
    delay_evaluation = 5, 
    evaluation_interval = 1
)

sweep_job.experiment_name="tunning"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=80, max_concurrent_trials=2, timeout=30200)

# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)